In [1]:
# configuration parameters
class Config:
    def __init__(self):
        self.INITIAL_POPULATION_SIZE = 10 # 150
        self.MAX_POPULATION_SIZE = 100
        self.INPUTS = 4
        self.OUTPUTS = 2
        self.MAX_GENERATIONS = 100
        
        self.MUTATE_WEIGHTS_PROPORTION = 0.75
        self.RANDOM_SEED = 42
        
CONFIG = Config()

import numpy as np
np.random.seed(CONFIG.RANDOM_SEED)


In [2]:
# TODO: incoming_edge can be a list of innovation numbers

NEURON_TYPES = ["input", "output", "hidden"]

class Neuron:
    NEURON_ID = 0
    
    def __init__(self, neuron_type = None):
        self.type = neuron_type
        if neuron_type == None:
            self.type = 'hidden'
            
        self.incoming_edges = []
        self.value = 0.0
        self.id = type(self).NEURON_ID
        type(self).NEURON_ID += 1
    
    def reset(self):
        self.value = 0.0
    
    def __repr__(self):
        return str(self.__dict__)

In [3]:
# Edge between 2 neurons

class Gene:
    INNOVATION_NUMBER = 0
    
    def __init__(self, input = 0, output = 0, weight = 0.0, enabled = True):
        self.input = input
        self.output = output
        self.weight = weight
        self.enabled = enabled
        self.innovation = type(self).INNOVATION_NUMBER
        type(self).INNOVATION_NUMBER += 1
    
    def __repr__(self):
        return str(self.__dict__)
    
    def mutate(self, epsilon = 1.0):
        self.weight = np.random.normal(self.weight, epsilon)

In [4]:
from math import exp
import pprint

# from config.py import *

# The neural network itself, consisting of neurons and edges (a.k.a genes)
class Genome:
    def __init__(self, neurons = [], genes = []):
        self.neurons = neurons
        self.genes = genes # list of edges
        self.fitness = 0
    
    def __repr__(self):
#         return str(self.__dict__)
        return pprint.PrettyPrinter(indent=4).pformat(self.__dict__)
    
    @staticmethod
    def generate():
        input_neurons = [Neuron('input') for _ in range(CONFIG.INPUTS)]
        output_neurons = [Neuron('output') for _ in range(CONFIG.OUTPUTS)]

        neurons = input_neurons + output_neurons
        
        genes = [Gene(input=input_neuron_index, output=CONFIG.INPUTS + output_neuron_index, weight=np.random.randn())
                 for input_neuron_index, _ in enumerate(input_neurons)
                 for output_neuron_index, _ in enumerate(output_neurons)]
        
        for output_neuron_index, output_neuron in enumerate(output_neurons):
            output_neuron.incoming_edges = [gene_index for gene_index, gene in enumerate(genes)
                                            if gene.output == output_neuron_index]

        # TODO: add random weights
            
        return Genome(neurons=neurons, genes=genes)
    
    @staticmethod
    def sigmoid(x):
#         return 2 / (1 + exp(-4.9 * x)) - 1
        return 1 / (1 + exp(-4.9 * x))

    def evaluate_neuron(self, neuron):
        if neuron.value != 0.0:
            return neuron.value
        
        neuron.value = self.sigmoid(sum(
            [self.genes[edge_index].weight * self.evaluate_neuron(self.neurons[self.genes[edge_index].input])
             for edge_index in neuron.incoming_edges
             if self.genes[edge_index].enabled]
        ))
        return neuron.value

    def activate(self, input_values):
        if len(input_values) != CONFIG.INPUTS:
            raise Error("invalid inputs length of {}".format(len(input_values)))
        
        input_index = 0
        for neuron in self.neurons:
            if neuron.type == 'input':
                neuron.value = input_values[input_index]
                input_index += 1
            else:
                neuron.reset()
        
        output_activations = [self.evaluate_neuron(neuron)
                              for neuron in self.neurons
                              if neuron.type == 'output']
        return output_activations
    
    def mutate_weights(self, proportion = CONFIG.MUTATE_WEIGHTS_PROPORTION):
        selected_genes = np.random.choice(self.genes, int(np.floor(len(self.genes) * proportion)))
        for gene in selected_genes:
            gene.mutate()
    
    # Mutation which adds a new neuron on an existing link between 2 other neurons
    def add_neuron(self):
        # Get a random gene (connection)
        gene = np.random.choice(self.genes)
        
        # take its input neuron
        input_neuron = self.neurons[gene.input]
        
        # take its output neuron
        output_neuron = self.neurons[gene.output]
        
        # create a new neuron
        new_neuron_index = len(self.neurons)
        new_neuron = Neuron()
        self.neurons.append(new_neuron)
        
        assert(self.neurons[len(self.neurons) - 1].id == new_neuron.id)
        
        # create a link from input to new
        link1_index = len(self.genes)
        link1 = Gene(input=gene.input, output=new_neuron_index, weight=1)
        self.genes.append(link1)
        
        # create a link from new to output
        link2_index = len(self.genes)
        link2 = Gene(input=new_neuron_index, output=gene.output, weight=gene.weight)
        self.genes.append(link2)

        # connect everything
        new_neuron.incoming_edges = [link1_index]
        output_neuron.incoming_edges.append(link2_index)
        
        # disable link from input to output
        gene.enabled = False

In [5]:
class Population:
    def __init__(self):
        self.genomes = [Genome.generate() for _ in range(CONFIG.INITIAL_POPULATION_SIZE)]
    
    def __repr__(self):
        return str(self.__dict__)

In [6]:
population = Population()

genome = population.genomes[0]
genome.activate([np.random.randn() for _ in range(CONFIG.INPUTS)])

[0.5, 0.5]

In [7]:
genome.mutate_weights()
genome

{   'fitness': 0,
    'genes': [   {'input': 0, 'output': 4, 'weight': 0.4967141530112327, 'enabled': True, 'innovation': 0},
                 {'input': 0, 'output': 5, 'weight': -0.13826430117118466, 'enabled': True, 'innovation': 1},
                 {'input': 1, 'output': 4, 'weight': 2.195103965605302, 'enabled': True, 'innovation': 2},
                 {'input': 1, 'output': 5, 'weight': 2.0244781879328304, 'enabled': True, 'innovation': 3},
                 {'input': 2, 'output': 4, 'weight': 0.9240332782086917, 'enabled': True, 'innovation': 4},
                 {'input': 2, 'output': 5, 'weight': -0.23413695694918055, 'enabled': True, 'innovation': 5},
                 {'input': 3, 'output': 4, 'weight': 1.8937257113705632, 'enabled': True, 'innovation': 6},
                 {'input': 3, 'output': 5, 'weight': 1.0245515992091883, 'enabled': True, 'innovation': 7}],
    'neurons': [   {'type': 'input', 'incoming_edges': [], 'value': -0.21967188783751193, 'id': 0},
              

In [8]:
genome.add_neuron()

In [9]:
genome

{   'fitness': 0,
    'genes': [   {'input': 0, 'output': 4, 'weight': 0.4967141530112327, 'enabled': True, 'innovation': 0},
                 {'input': 0, 'output': 5, 'weight': -0.13826430117118466, 'enabled': True, 'innovation': 1},
                 {'input': 1, 'output': 4, 'weight': 2.195103965605302, 'enabled': True, 'innovation': 2},
                 {'input': 1, 'output': 5, 'weight': 2.0244781879328304, 'enabled': False, 'innovation': 3},
                 {'input': 2, 'output': 4, 'weight': 0.9240332782086917, 'enabled': True, 'innovation': 4},
                 {'input': 2, 'output': 5, 'weight': -0.23413695694918055, 'enabled': True, 'innovation': 5},
                 {'input': 3, 'output': 4, 'weight': 1.8937257113705632, 'enabled': True, 'innovation': 6},
                 {'input': 3, 'output': 5, 'weight': 1.0245515992091883, 'enabled': True, 'innovation': 7},
                 {'input': 1, 'output': 6, 'weight': 1, 'enabled': True, 'innovation': 80},
                 {'inp

In [10]:
# n1 = Neuron('hidden', [], 1)
# n1.value

np.random.normal(10, 1)
np.random.choice([1, 2, 3], 2)

array([1, 1])

In [16]:
from operator import itemgetter
from copy import deepcopy
import gym
import time

# https://github.com/openai/gym/wiki/CartPole-v0
env = gym.make('CartPole-v0')

# print(gym.envs.registry.all())

population = Population()

def one_generation(render):
    for genome in population.genomes:
        observation = env.reset()
        done = False
        lifespan = 0

        genome.mutate_weights()
        
        while not done:
            lifespan = lifespan + 1
            if render:
                env.render()
            #print(observation)
            activation = genome.activate(observation)
            action = np.argmax(activation)
            observation, reward, done, info = env.step(action)
            #print("Reward: {}".format(reward))
            if done:
                genome.fitness = lifespan / len(genome.genes)
                print("Episode finished after {} timestamps".format(lifespan))
    
    population.genomes.sort(key=lambda genome: genome.fitness, reverse=True)
    print(list(map(lambda genome: genome.fitness, population.genomes)))
    
    for genome in population.genomes[:2]:
        copy = deepcopy(genome)
        copy.add_neuron()
        population.genomes.append(copy)
    

for generation in range(CONFIG.MAX_GENERATIONS):
    print("Genration: {}".format(generation))
    one_generation(render=generation == CONFIG.MAX_GENERATIONS - 1)
    print()

env.close()

population

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Genration: 0
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 8 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode finished after 11 timestamps
[1.375, 1.25, 1.25, 1.25, 1.125, 1.125, 1.125, 1.125, 1.125, 1.0]

Genration: 1
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 8 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode finished after 10 timestamps
[1.2

Episode finished after 9 timestamps
Episode finished after 8 timestamps
Episode finished after 10 timestamps
[1.4, 1.25, 1.125, 1.125, 1.125, 1.125, 1.125, 1.125, 1.125, 1.125, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8]

Genration: 18
Episode finished after 12 timestamps
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 11 timestamps
Episode finished after 9 timestamps
Episode finished after 10 timestamps
Episode finished after 11 timestamps
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finis

Episode finished after 10 timestamps
Episode finished after 11 timestamps
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 11 timestamps
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode finished after 11 timestamps
Episode finished after 8 timestamps
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode finished after 8 timestamps
Episode finished after 9 timestamps
Episode finished after 11 timestamps
Episode finished after 9 timestamps
Episode finished after 8 timestamps
Episode finished after 8 timestamps
Episode finished after 9 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finishe

Episode finished after 200 timestamps
Episode finished after 116 timestamps
Episode finished after 127 timestamps
Episode finished after 67 timestamps
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 11 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode finished after 11 timestamps
Episode finished after 9 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 8 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode fin

Episode finished after 132 timestamps
Episode finished after 200 timestamps
Episode finished after 200 timestamps
Episode finished after 109 timestamps
Episode finished after 200 timestamps
Episode finished after 21 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 8 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 13 timestamps
Episode finished after 9 timestamps
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode finished after 10 timestamps
Episode finished after 10 timestamps
Episode finished after 9 timestamps
Episode finished after 9 timestamps
Episode finished after 10 timestamps
Epis

...

Episode finished after 186 timestamps
Episode finished after 164 timestamps
Episode finished after 119 timestamps
Episode finished after 165 timestamps
Episode finished after 130 timestamps
Episode finished after 200 timestamps
Episode finished after 129 timestamps
Episode finished after 48 timestamps
Episode finished after 200 timestamps
Episode finished after 122 timestamps
Episode finished after 167 timestamps
Episode finished after 163 timestamps
Episode finished after 169 timestamps
Episode finished after 11 timestamps
Episode finished after 114 timestamps
Episode finished after 66 timestamps
Episode finished after 63 timestamps
Episode finished after 200 timestamps
Episode finished after 122 timestamps
Episode finished after 149 timestamps
Episode finished after 8 timestamps
Episode finished after 152 timestamps
Episode finished after 9 timestamps
Episode finished after 200 timestamps
Episode finished after 84 timestamps
Episode finished after 87 timestamps
Episode finished after

Episode finished after 200 timestamps
Episode finished after 200 timestamps
Episode finished after 170 timestamps
Episode finished after 98 timestamps
Episode finished after 162 timestamps
Episode finished after 174 timestamps
Episode finished after 200 timestamps
Episode finished after 151 timestamps
Episode finished after 200 timestamps
Episode finished after 166 timestamps
Episode finished after 144 timestamps
Episode finished after 200 timestamps
Episode finished after 200 timestamps
Episode finished after 146 timestamps
Episode finished after 111 timestamps
Episode finished after 200 timestamps
Episode finished after 40 timestamps
Episode finished after 31 timestamps
Episode finished after 151 timestamps
Episode finished after 144 timestamps
Episode finished after 69 timestamps
Episode finished after 44 timestamps
Episode finished after 9 timestamps
Episode finished after 44 timestamps
Episode finished after 74 timestamps
Episode finished after 67 timestamps
Episode finished after

Episode finished after 193 timestamps
Episode finished after 31 timestamps
Episode finished after 196 timestamps
Episode finished after 32 timestamps
Episode finished after 200 timestamps
Episode finished after 19 timestamps
Episode finished after 200 timestamps
Episode finished after 200 timestamps
Episode finished after 161 timestamps
Episode finished after 200 timestamps
Episode finished after 104 timestamps
Episode finished after 51 timestamps
Episode finished after 200 timestamps
Episode finished after 200 timestamps
Episode finished after 154 timestamps
Episode finished after 146 timestamps
Episode finished after 200 timestamps
Episode finished after 200 timestamps
Episode finished after 200 timestamps
Episode finished after 180 timestamps
Episode finished after 200 timestamps
Episode finished after 167 timestamps
Episode finished after 187 timestamps
Episode finished after 144 timestamps
Episode finished after 195 timestamps
Episode finished after 200 timestamps
Episode finished

Episode finished after 200 timestamps


KeyboardInterrupt: 